FIVE DETECTOR MODEL, LOOKS LIKE A PLATEAU, SEE WRITE-UP FOR DRAWING

In [ ]:
%matplotlib notebook

import time
import numpy as np
import matplotlib.pyplot as plt
import math as mth
import random as rand
import statistics as s
import healpy as hp
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from mpl_toolkits.basemap import Basemap
from matplotlib import cm, colors


In [ ]:
def length(v):
    return mth.sqrt(np.dot(v, v))

def angle(v1, v2):

    return mth.acos(np.dot(v1, v2) / (length(v1) * length(v2)))

def frodo(theta):
    #getting the weighted average of the area of various parts along the unit sphere. Current simulation targets azimuth many more times than the rest of the sphere, so have to account for this being in a significantly smaller area
    #keep in mind this 5 has to do with the # of test theta, 5 works best since it is splits the rings perfectly between evenly spaced points of 10 (used)
    if theta==0: 
        #have to treat this slightly differently
        A=abs(2*np.pi*(np.cos(theta)-np.cos(np.deg2rad(5)))) 
    else:
        A=abs(2*np.pi*(np.cos(theta+np.deg2rad(5))-np.cos(theta-np.deg2rad(5))))
    return A

def chimaker(chiterms,Ndets): 
    #separate out all of the individual chi squared terms, and add each together. 
    a,b,c,d,e=np.array_split(chiterms,Ndets)
    ab=np.add(a,b)
    cd=np.add(c,d)
    ad=np.add(ab,cd)
    chisquareds=(np.add(ad,e)) 
    return chisquareds

def solver(detsvals,bottheta,toptheta,botphi,topphi,n):
#this function uses a chi squared minimizer over a given range to identify the theta,phi,
#and Ao values which correspond to the minimum chi squared and thus localized source. 
    confidence = []
    chiterms = []
    thecon = []
    phicon = []
    for s in range(len(detsvals)):     
        oa=np.deg2rad(np.linspace(bottheta,toptheta,n/4))  #range of thetas to sample
        ob=np.deg2rad(np.linspace(botphi,topphi,n)) #phi
        Aofit=np.linspace(0,1000,25)  
        for sa in range(len(oa)): 
            for sb in range(len(ob)):
                for sc in range(len(Aofit)):
                    #make sure it fits within the acceptable range
                    if oa[sa]>=0 and oa[sa]<=np.pi and ob[sb]>=0 and ob[sb]<=2*np.pi:
                        CHIsourceang=[oa[sa],ob[sb]]
                        CHIsourcexyz = hp.ang2vec(CHIsourceang[0],CHIsourceang[1])
                        CHIsep=angle(CHIsourcexyz,dets[s])                                           
                        if CHIsep<np.pi: 
                            chi=Aofit[sc]*np.dot(CHIsourcexyz,dets[s])+bg
                        else:
                            chi=0            
                        if detvals[s]>0:   #if there is a signal in the detector 
                            chiterm=((chi-detsvals[s])**2/detsvals[s])
                        else:    #if not, just zero 
                            chiterm=10000
                    else: 
                        chiterm=10000 #some large #, just to note that it definitely isn't right angle 
                                                                       
                                    
                    chiterms.append(chiterm)   #this is an array of EVERY SINGLE term, need to split in pieces and add element by element...
    chisquareds=chimaker(chiterms,len(detsvals))
    
    chimin=np.amin(chisquareds)
    chisquareds=list(chisquareds)
    thetaloc = np.rad2deg(oa[int((chisquareds.index(chimin)-(chisquareds.index(chimin) % (len(ob)*len(Aofit))))/(len(ob)*len(Aofit)))])
    philoc = np.rad2deg(ob[int(((chisquareds.index(chimin) % (len(ob)*len(Aofit)))-(chisquareds.index(chimin) % (len(Aofit))))/len(Aofit))])
    Aoguess=Aofit[int((chisquareds.index(chimin) % (len(ob)*len(Aofit)))  % len(Aofit))]
    #print(Aoguess)
    #print(thetaloc,philoc)
    
    return thetaloc,philoc,chisquareds,chimin,Aoguess

In [ ]:
start_time = time.time()

zenith=[0,0]  #set zenith of the system

#Test detector orientations(letters B,C,E,etc. are at same but oppiste side angles)
tiltA=np.deg2rad(np.array([15,30,45,75]))    #linspace(30,75,4))
tiltD=np.deg2rad(np.array([15,30,45,75]))    #linspace(30,75,4)

#Test source locations
thetaS=np.deg2rad(np.linspace(0,60,7))          #  #currently set at the zenith , can change this in one direction but not both simultaneously (yet)
phiS=np.deg2rad(np.linspace(0,360,16))     #array([180]))   #these are off


Ao=np.array([500]) #intensity of the signal, can loop through more! 
bg=1000   #background noise in each detector
error=[]
x = 10   #run x times per source location
y=1 #whatever degree to within confidence
delta=5  #pinpointing in coarse-fine localization
chimins = []   #create a list of every minimized chisquareds, good for comparing if needed
RMSs = []
for ja in range(len(tiltA)):   #run through the possible detector orientations
    #should include another loop starting here for the other tilt direciton (D)
    tiltB=0
    tiltC=tiltA[ja]
    for jb in range(len(tiltD)): 
        print("Detector Tilt " + str(np.rad2deg(tiltA[ja]))+ " degrees x " +str(np.rad2deg(tiltD[jb]))+ " degrees")
        tiltE=tiltD[jb]
        for k in range(len(Ao)):  #(optional) run through different signal strengths
            angRMS = []   #rms vals for every theta, phi 
            totarea=0 
            azavg = []  #average RMS within each azimuth position, aka within a ring 
            ringareas = []   #all of the areas of the rings
            for ia in range(len(thetaS)):  #run through the possible source angles
                #area of ring here 
                ringsize = frodo(thetaS[ia])  
                ringareas.append(ringsize)
                totarea += ringsize  #don't really need this, just to confirm it's doing the right thing!
                #print("Ring area and total cumulative area" + str(ringsize) + " " +str(totarea))
                ringmeansep = 0  #resets for each
                for ib in range(len(phiS)):
                    Aoguesses = []
                    angseps = []
                    sourceang=[thetaS[ia],phiS[ib]]
                    #print("testing ... "+ str(np.rad2deg(sourceang)))

                    sourcexyz = hp.ang2vec(sourceang[0],sourceang[1])
                    Anorm = hp.ang2vec(zenith[0]+tiltA[ja],zenith[1])
                    Bnorm = hp.ang2vec(zenith[0]+tiltB,zenith[1])
                    Cnorm = hp.ang2vec(zenith[0]+tiltC,zenith[1]+np.pi)
                    Dnorm = hp.ang2vec(zenith[0]+tiltD[jb],zenith[1]+np.pi/2)
                    Enorm = hp.ang2vec(zenith[0]+tiltE,zenith[1]+1.5*np.pi)
                    #add anymore if want to, one idea is to add 4 more for the edges btw the CAD and BCD bounds,
                    #will have 9 detectors
                    
                    dets = [Anorm,Bnorm,Cnorm,Dnorm,Enorm]   #and whatever other detectors to be included!
                    count=0
                    while count<x: 
                        detvals = []
                        for ii in range(len(dets)):
                            sep=angle(sourcexyz,dets[ii])
                            if sep<np.pi:
                                dexp=Ao[k]*np.dot(sourcexyz,dets[ii])  #expected number of counts in A
                            else:
                                dexp=0
                                #print("Warning: expected signal not found in detector "+str(ii))
                            counts=max(dexp,0)+bg
                            unccounts=np.sqrt(counts)
                            det=rand.gauss(counts,unccounts)
                            if det-bg<0:
                                det=0
                                #print("Warning: no signal found in detector "+str(ii))
                            #talking too much, had to mute
                            detvals.append(det)  #each detector with noise, need this array for solver
                        coarsethetaloc,coarsephiloc,coarsechisquareds,cmin,coarseAo = solver(detvals,0,90,0,360,20)  #runs through big range with few points, easy to pinpoint from here
                        finethetaloc,finephiloc,finechisquareds,fmin,fineAo = solver(detvals,coarsethetaloc-delta,coarsethetaloc+delta,coarsephiloc-delta,coarsephiloc+delta,12)                      
                        chimins.append(fmin)
                        
                        Aoguesses.append(fineAo)
                        acvec=hp.ang2vec(np.deg2rad(finethetaloc),np.deg2rad(finephiloc)) #actual vector position (on unit circle)  of the source for trial x-n
                        angsep = angle(sourcexyz,acvec)
                        angseps.append(angsep)   #array of how far apart they
                        #print(np.rad2deg(angsep))
                        count += 1     
                    STDangsep = np.std(angseps)  #,thetaS[ia])
                    meansep=s.mean(np.rad2deg(angseps))
                    ringmeansep+=meansep  #reset this before every theta
                    angRMS.append(meansep)
                    
                    #scatter color plots commented out below 
                    #plt.scatter(np.rad2deg(sourceang[1]),np.rad2deg(sourceang[0]),c=np.rad2deg(STDangsep),s=100,cmap='viridis_r',vmin=0, vmax=15)
                   
                    #print("std dev = " + str(np.rad2deg(STDangsep)) + " deg")
           
            #azimuthal average of each ring
                azavg.append(ringmeansep/len(phiS))
                #is ringSTD azimuthal average? let's double check. 
                #print((ringSTD))
        #so here I have all the stuff done, now I need to weight each point 
                print("starting the next theta...")
        
        weightedRMS = []    
        for il in range(len(azavg)):         
        #multiply each number by its weight, divide by its total sum, add up
            weightedRMS.append(azavg[il]*ringareas[il])
        print(np.sum(weightedRMS)/np.sum(ringareas))
        RMSs.append(np.sum(weightedRMS)/np.sum(ringareas)) 
           # for every test source in every ring for every theta, average these out to find a weighted avg of how valuable they are
            
            
            #prints out the standard deviation at each tilt, and has it weighted based on how large of an area this represents       
end_time = time.time()
print(RMSs)
print("Time elapsed: "+ str(end_time-start_time))

In [ ]:
s.mean(RMSs)